# A program returning those informations given a "dests.txt" file:
1.	Distance of the destination from Jerusalem in kilometers.
2.	Arrival time from Jerusalem to the destination in hours and minutes.
3.	Longitude of the destination
4.	Latitude of the destination

Using 2 functions get_dist_jer(gets the distance and the time from Jerusalem) and get_lng_lat(gets the lng and lat of the destination)
get_dict() as the main function returning a dictionary with the details

In [ ]:
import requests
import json
from typing import Tuple, List, Dict


def get_dist_jer(origin: str, api_key: str) -> Tuple[str, str]:
    url = "https://maps.googleapis.com/maps/api/distancematrix/json?destinations=Jerusalem&origins=%s&key=%s&units=metrics" % (
        origin, api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error", response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
        print("requests.get doesn't work properly")

    city_distance = response_data['rows'][0]['elements'][0]['distance']['text']
    city_travel_time = response_data['rows'][0]['elements'][0]['duration']['text']
    
    # It wasn't clear in the assignement if we need to return the time in minutes or in hours and minutes, so this is the minutes option
    # raw_city_travel_time = response_data['rows'][0]['elements'][0]['duration']['value']
    # city_travel_time = round(int(raw_city_travel_time) / 60)
    
    return city_distance, city_travel_time


def get_lng_lat(city_name: str, api_key: str) -> Tuple[str, str]:
    url = "https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (
        city_name, api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error", response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
        print("requests.get doesn't work properly")

    city_lng = response_data['results'][0]['geometry']['location']['lng']
    city_lat = response_data['results'][0]['geometry']['location']['lat']

    return city_lng, city_lat


def get_dict() -> Dict:
    api_key = '{My_api_key}'
    city_list = open("dests.txt").read().splitlines()
    city_dict = dict()
    for city in city_list:
        details_list = [get_dist_jer(city, api_key)]
        details_list.append(get_lng_lat(city, api_key))
        formated_list = str(details_list).replace("(", "").replace(")", "")
        city_dict[city] = formated_list
    return city_dict


All the informations about each destination present in dests.txt file

In [14]:
from exercise4 import get_dict

get_dict()

{'Istanbul': "['1,787 km', '20 hours 44 mins', 28.9783589, 41.0082376]",
 'Amsterdam': "['4,504 km', '2 days 0 hours', 4.9041389, 52.3675734]",
 'Valletta': "['3,769 km', '2 days 3 hours', 14.5145528, 35.8989085]",
 'Basel': "['4,067 km', '1 day 20 hours', 7.5885761, 47.5595986]",
 'Doha': "['2,109 km', '22 hours 1 min', 51.53103979999999, 25.2854473]"}

Given get_dict dictionary, returns a list of the 3 cities furthest from Jerusalem

In [18]:
from typing import List

def find_long_dist() -> List:
    city_dict = get_dict()
    city_list = []
    new_dict = dict()
    for city in city_dict:
        raw_data = str(city_dict[city]).split(' km')[0]
        city_dist = raw_data.replace("[", "").replace("'", "")
        new_dict[city] = city_dist
    sorted_dict = sorted(new_dict.items(), key=lambda city: city[1], reverse=True)
    furt_details = (sorted_dict[0:3])
    furt_cities = [x[0] for x in furt_details]
    return furt_cities

find_long_dist()

['Amsterdam', 'Basel', 'Valletta']